# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\\
& \;\; 
f \leq u
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. TO DO }
\\
&\text{2. }
\end{align}


#### Remarks
First, we tried to solve the STA with CVX.
This does not work because the values of the travel time are to big. 

Secondly, we solve the STA using links flow. 
This does not help us in our project because we need the path flow to compute the dual of the TAP-C.

Thirdly, we solve the STA using paths flow. 
The issue here is that finding every path in a network in a NP-complete problem. 
But if we can compute every path (we only need to do it one time), then it is really fast to compute the shortest path. We do not need to do a Dikjstra's algorithm to compute the shortest path, only a big matrix multiplication and a array sorting are enough. This might be more difficult on a laptop. But it might be faster on a HPC.

Finally, we solve the STA using links flow and we compute the paths during the all or nothing step of the Frank Wolf algorithm.

TO DO: we should be carefull that all function does not use global variables.

In [84]:
import numpy as np
import scipy.sparse

In [85]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network = I210
debug = True

## 1. We load the graph and the demand
Both graph and demand are in csv file, we load them

In [86]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

### We clean the data

In [87]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a0, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1,
    and a0, ..., a4 are the coeficient to calculate the travel time of one link as a function
    of the flow on the link: t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4

One can add some checks here to be sure that demand and graph respect this format!

Some demand and graph data can be found on:
    https://github.com/bstabler/TransportationNetworks
    Here one need to perprocess the data before using this code
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

graph, demand = cleaning_input(graph, demand)
if network == Brae:
    demand[0][2] = 10
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
nb_ods = int(demand.shape[0])
if debug:
    print("nb nodes = " + str(nb_nodes))
    print("nb links = " + str(nb_links))
    print("nb ods = " + str(nb_ods))

nb nodes = 20
nb links = 41
nb ods = 1


Then, we define the function which gives the travel time as a function of the flow

In [88]:
## Here Max float is fixed to not overlap the maximum float number when we compute the potential function
max_float = 1e+10
if debug:
    print("The max capacity and to max travel time are " + str(max_float))

"""
We define the travel time of each link as a function of the flow on each link.
This function is given by the network topology (in the graph file) and by the capacity of each link.
One row of the graph is:
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4]
Each row correspond to one link.
The travel time of the link is t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4 if f < c 
    It is t = + inf (max_float) is f >= c
"""
def travel_time(graph, f, c = -1):
    if c == -1:
        c = [max_float for i in range(len(f))]
    # here we need to have the same indexation for graph and f. That why we need to store graph in a dictionnary
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else max_float for i in range(len(f))]
    return tt_tmp

The max capacity and to max travel time are 10000000000.0


## 2. We compute the all or nothing flow allocation

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [89]:
"""
We store the travel time of each link in a sparce adjacency matrix G.
Given a flow allocation f and a capacities constraints c, we compute 
the travel time of each link and store it in G.

This allow us after to us the Disjtra's algorithm of the class scipy.sparse.csgraph
"""

def update_travel_time_from_tt(graph, tt):
    G = np.zeros(shape=(nb_nodes,nb_nodes))
    G = scipy.sparse.lil_matrix(G)
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    G = G.tocsr()
    return G

def update_travel_time_from_flow(graph, f, c=-1):
    return update_travel_time_from_tt(graph, travel_time(graph, f, c))

In [94]:
G = update_travel_time_from_flow(graph, np.zeros(nb_links)) # , [1,1,-1,1,1]))
debug_local = False
if debug_local:
    print(G)

In [95]:
from scipy.sparse.csgraph import dijkstra

In [96]:
"""
Because the classic Frank Wolf algorithm does not take into account the paths,
we need to store the path in memory. We do it by using Object and a dictionnary 
called paths_used.

This idea is that we do not want to compute all the possible paths of the network
before the beginning of the algorithm. So we compute the paths used during the algorithm,
and we store them in memory.

We build at the same time the incidence matrix.

CAREFUL. We use the number of links of the graph in the calcul of the hash of every path.
This variable should be a parameter.
"""

class path:
    def __init__(self,links):
        self.links = links
        self.__flow = 0
    def set_flow(self, flow):
        self.__flow = flow
    def get_flow(self,):
        return self.__flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    # I am not sure about the hash table structure in Python. I am used to Java.
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.__flow) + " is on " + str(self.links) 

debug_local = False
if debug_local:
    paths_used = {}
    print(paths_used)
    p = path([0, 1, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 1, 1, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 0, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)

Now let's compute the all or nothing allocation

In [97]:
"""
This cell is the main point of the entire solver.
It is the all or nothing allocation.

"""

# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])

# computing the all or nothing flow
def all_or_nothing(demand, G, graph_dict, paths_used, k, full_dijkstra = True):
    # k is the next index to give to a new path.
    debug_local = 0
    # paths_used_tmp is the set of the path in the all or nothing allocation of this step
    paths_used_tmp = {}
    # faon is the flow allocation of the all or nothing allocation
    faon = np.zeros(nb_links) # nb_links should be a paramter.
    
    # using scipy to compute dijkstra
    """
    HERE ONE CAN CHANGE THE ALGORITHM TO BE FASTER. WE MIGHT REWRITE THE DIJKSTRA ALGORITHM.
    indices : array_like or int, optional
        if specified, only compute the paths for the points at the given indices.
    """
    indices = None
    if not full_dijkstra:
        indices = [int(demand[i,0]) for i in range(len(demand))]
        
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True, indices = indices)
    if debug_local:
        print(return_predecessors)
        print(indices)
        
    # for every origin destination pairs (i.e. one line of the demand file)
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp = d_tmp
        links = []
        # using the dijkstra, we build the fastest path and we put the flow on it.
        while node_tmp != o_tmp:
            if debug_local:
                print(o_tmp)
                print(node_tmp)
            if not full_dijkstra:
                node_tmp_d = return_predecessors[i][node_tmp]
            else:
                node_tmp_d = return_predecessors[o_tmp][node_tmp]
            # Here we need the graph_dict to recover the link id from the nodes id.
            link_tmp = graph_dict[node_tmp_d][node_tmp]
            # we recover the path from the predecessor of the 
            links.insert(0, link_tmp)
            faon[link_tmp] += flow_tmp
            node_tmp = node_tmp_d
        
        p = path(links)
        p.set_flow(flow_tmp)
        
        
        if debug_local:
            print(p)
        # If we do not already know p, we add it to the delta matrix (here a dict)
        if not hash(p) in paths_used:
            if debug_local:
                print(k)
            # we add p and his index
            paths_used[hash(p)] = (p, k)
            paths_used_tmp[hash(p)] = (p, k)
            k = k+1
        else:
            # we recover the index of p using the paths_used dict
            paths_used_tmp[hash(p)] = (p, paths_used[hash(p)][1])
    return faon, paths_used_tmp, paths_used, k

We define the line search

In [99]:
paths_used = {}
debug_local = False
if debug_local:
    k = 0 
    _, _ , pp, k = all_or_nothing(demand, G, graph_dict, paths_used, k)
    for p in pp.values():
        print(p[0])
    all_or_nothing(demand, G, graph_dict, paths_used, k, False)
    for p in paths_used.values():
        print(p[0])

In [100]:
"""
The function potential is used to compute the line search between 
the all or nothing allocation and the current flow allocation
The function potential returns the objective function corresponding to
the flow allocation f.

The function line search does a 1D line search.
"""

def potential(graph, f, c=-1):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    pot_tmp = graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5)
    pot_tmp = [pot_tmp[i] if f[i]<c[i] else f[i]*max_float for i in range(len(f))]
    return np.sum(pot_tmp)


def line_search(f, res=20):
    debug_local = False
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if debug_local:
            print(l * d, end=": ")
            print(f(l * d))
            print(r * d, end=": ")
            print(f(r * d))
            print(str(m1 * d) + " = " + str(f(m1 * d)))
            print(str(m2 * d) + " = " + str(f(m2 * d)))
            print()
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [118]:
def Frank_Wolf_solver(graph, eps, nb_iter):
    k = 0
    all_paths_used = {}
    G = update_travel_time_from_flow(graph, np.zeros(nb_links))

    # The initialization step: we put all the demand on the fastest free flow travel time paths.
    f, paths_used_for_this_iter, all_paths_used, k = all_or_nothing(demand, G, graph_dict, all_paths_used, k)
    G = update_travel_time_from_flow(graph, f)
    path_flow_matrix = np.zeros(k)
    for val in paths_used_for_this_iter.values():
        path_flow_matrix[val[1]] = val[0].get_flow()

    if debug:
        print(f)
        for p in paths_used_for_this_iter.values():
            print(p[0])
        print(path_flow_matrix)

    # The iteration of the Frank-Wolf algorithm
    for i in range(nb_iter):
        faon, paths_used_for_this_iter, all_paths_used, k = all_or_nothing(demand, G, graph_dict, all_paths_used, k)

        # we find the better convex combinaison of f and faon
        s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
        f = (1. - s) * f + s * faon
        G = update_travel_time_from_flow(graph, f)

        # we multiply the previous path flow matrix by the coeficient of the line search
        path_flow_matrix = (1-s) * path_flow_matrix
        # we add the new path at the end of the path flow matrix.
        path_flow_matrix = np.append(path_flow_matrix, np.zeros(len(all_paths_used)-path_flow_matrix.shape[0]))
        # we add the all or nothing path flow (mulitply by the coeficient of the line search) to the path flow matrix.
        for val in paths_used_for_this_iter.values():
            # val[1] is the index of the path, val[0] is the path object
            path_flow_matrix[val[1]] += s * val[0].get_flow()

        if debug and i % (nb_iter / 10) == 0:
            print("Iteration: " + str(i))
            print("s: " + str(s))
            print("The paths used at this iteration are:")
            for p in paths_used_for_this_iter.values():
                print(p[0])
        if s < eps:
            break
    if debug:
        print(path_flow_matrix)
       
    # At the end we do some work to return a proper output
    nb_paths = len(all_paths_used.keys())
    delta = np.zeros(shape=(nb_paths, nb_links))
    tt_f = np.array(travel_time(graph, f))

    for p in paths_used.keys():
        links_tmp = paths_used[p][0].links
        for l in links_tmp:
            delta[paths_used[p][1]][l] = 1
    print(len(delta))
    print(len(delta[0]))
    print(len(tt_f))
    delta_sparse = scipy.sparse.csr_matrix(delta)

    if debug:
        print(nb_paths)
        print(tt_f)
        print(delta)
        print(delta_sparse @ tt_f)

eps=1e-8
nb_iter = 1000
Frank_Wolf_solver(graph, eps, nb_iter)

[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0. 25000.     0.     0. 25000.     0.     0. 25000.     0.
     0. 25000.     0.     0. 25000.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0. 25000.
 25000.]
25000.0 is on [40, 12, 15, 18, 21, 24, 39]
[25000.]
Iteration: 0
s: 0.13017301136669227
The paths used at this iteration are:
25000.0 is on [40, 13, 29, 31, 33, 35, 37, 38, 39]
Iteration: 100
s: 7.796287695782818e-05
The paths used at this iteration are:
25000.0 is on [40, 12, 15, 18, 22, 35, 36, 24, 39]
[1.89522781e+04 2.70002298e+03 2.58494790e+03 1.79501481e+02
 8.98800319e+01 4.61496980e+01 2.55162433e+01 2.34910714e+01
 3.87494869e+01 1.06487565e+01 3.41884806e+01 1.79223420e+01
 1.62334098e+01 1.86106002e+01 3.09289082e+01 1.17248706e+01
 1.31311598e+01 1.22192118e+01 1.12021600e+01 8.85574914e+00
 1.52706640e+01 1.00187094e+01 4.71225851e+00 1.43751400e+01
 9.24551061e+00 1.25396841e+01 3.02

In [23]:
for p in paths_used.values():
    index = p[1]
    p[0].set_flow(path_matrix[index])
    print(str(p[0]))
if debug:
    print(demand)
    print(G)
    print(f)



18952.278101367956 is on [40, 12, 15, 18, 21, 24, 39]
2700.022984741186 is on [40, 13, 29, 31, 33, 35, 37, 38, 39]
2584.9479006483425 is on [40, 11, 0, 2, 4, 6, 8, 10, 39]
179.5014807712316 is on [40, 12, 14, 2, 4, 6, 9, 24, 39]
89.88003194795711 is on [40, 12, 16, 31, 33, 35, 36, 24, 39]
46.149697992468255 is on [40, 12, 14, 2, 4, 6, 9, 25, 37, 38, 39]
25.516243308277954 is on [40, 13, 29, 31, 33, 35, 36, 24, 39]
23.491071394909238 is on [40, 12, 14, 2, 4, 7, 21, 24, 39]
38.749486881887 is on [40, 12, 16, 31, 32, 18, 21, 24, 39]
10.648756537575332 is on [40, 12, 15, 19, 33, 35, 37, 38, 39]
34.18848060110483 is on [40, 12, 15, 18, 20, 6, 9, 24, 39]
17.922341975316343 is on [40, 13, 29, 30, 14, 2, 5, 19, 33, 35, 37, 38, 39]
16.233409782801882 is on [40, 12, 16, 31, 32, 17, 4, 7, 21, 23, 8, 10, 39]
18.61060015818257 is on [40, 12, 14, 2, 5, 19, 33, 34, 21, 24, 39]
30.92890817872998 is on [40, 12, 15, 18, 22, 35, 36, 24, 39]
11.724870624654828 is on [40, 12, 15, 17, 4, 7, 21, 23, 8, 10, 3

In [24]:
print("[", end="")
for i in range(len(travel_time(graph, f))):
    print(travel_time(graph, f)[i], end=", ")
print()

[147875935652647.06, 1.15, 246559669925329.97, 1.14, 298508087749981.44, 104677361.5427441, 221682960670049.2, 175988031.85122514, 285479656274557.8, 21169223740.42079, 138299033484119.11, 123229946377205.9, 270765706353557.0, 56019919334891.664, 42203103541.35713, 246598725846519.3, 1005103366.5454719, 84770679.48491268, 298489150269862.06, 42975542.30257825, 126596872.42104061, 221688297296678.72, 16607433.11956949, 74234854.10581149, 423778200910363.56, 145535092.5306741, 1.09, 0.47, 0.36, 214743024117084.7, 40622935.117374435, 246603728865090.53, 52424578.198410496, 298491557980177.6, 18051027.0646503, 221699239115847.7, 1581550565.2371945, 349734374575524.8, 74042863085809.28, 330000000000000.25, 435000000000000.3, 


In [25]:
print(graph)
print(paths_used)

[[0.0000e+00 0.0000e+00 1.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [1.0000e+00 0.0000e+00 6.0000e+00 1.1500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7600e+00]
 [2.0000e+00 1.0000e+00 2.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [3.0000e+00 1.0000e+00 7.0000e+00 1.1400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7360e+00]
 [4.0000e+00 2.0000e+00 3.0000e+00 1.6800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 4.0320e+00]
 [5.0000e+00 2.0000e+00 8.0000e+00 1.1300e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7120e+00]
 [6.0000e+00 3.0000e+00 4.0000e+00 1.2600e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.0240e+00]
 [7.0000e+00 3.0000e+00 9.0000e+00 1.0500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.5200e+00]
 [8.0000e+00 4.0000e+00 5.0000e+00 2.2500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 5.4000e+00]
 [9.0000e+00 4.0000e+00 1.0000e+01 1.0400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.4960e+00]
 [1.0000e+01 5.0000e+00 1.1000e+01 1.0900e+00 0.0000e+00 0.0000e+00
  

In [26]:
"""
TO DO: ADD THE CORRECT OUTPUT
CONSTRUCT THE DELTA MATRIX AND THE ROUTE2OD FROM THE paths_used
"""
nb_paths = len(paths_used.keys())
delta = np.zeros(shape=(nb_paths, nb_links))
tt_f = np.array(travel_time(graph, f))

for p in paths_used.keys():
    links_tmp = paths_used[p][0].links
    for l in links_tmp:
        delta[paths_used[p][1]][l] = 1
print(len(delta))
print(len(delta[0]))
print(len(tt_f))
delta_sparse = scipy.sparse.csr_matrix(delta)

if debug:
    print(nb_paths)
    print(tt_f)
    print(delta)
    print(delta_sparse @ tt_f)

74
41
41
74
[1.47875936e+14 1.15000000e+00 2.46559670e+14 1.14000000e+00
 2.98508088e+14 1.04677362e+08 2.21682961e+14 1.75988032e+08
 2.85479656e+14 2.11692237e+10 1.38299033e+14 1.23229946e+14
 2.70765706e+14 5.60199193e+13 4.22031035e+10 2.46598726e+14
 1.00510337e+09 8.47706795e+07 2.98489150e+14 4.29755423e+07
 1.26596872e+08 2.21688297e+14 1.66074331e+07 7.42348541e+07
 4.23778201e+14 1.45535093e+08 1.09000000e+00 4.70000000e-01
 3.60000000e-01 2.14743024e+14 4.06229351e+07 2.46603729e+14
 5.24245782e+07 2.98491558e+14 1.80510271e+07 2.21699239e+14
 1.58155057e+09 3.49734375e+14 7.40428631e+13 3.30000000e+14
 4.35000000e+14]
[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [1. 0. 1. ... 0. 1. 1.]
 ...
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]]
[2.22632008e+15 2.22633471e+15 2.22663529e+15 2.22635800e+15
 2.22634102e+15 2.22635718e+15 2.22633725e+15 2.22634234e+15
 2.22632614e+15 2.22633251e+15 2.22633604e+15 2.22633304e+15
 2.22634590e+15 2.2263